## Baseline using Yolov8n


In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00


In [2]:
%pwd

'/kaggle/working'

In [3]:
from ultralytics import YOLO
import cv2

model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture("../crowd.mp4")
ret, frame = cap.read()
if ret:
    results = model(frame)
    annotated = results[0].plot()  # ← стандартная отрисовка YOLO
    cv2.imwrite("test_frame.jpg", annotated)
cap.release()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


видно что есть лишние объекты и слишком жирная рамка

In [4]:
import cv2

cap = cv2.VideoCapture("../crowd.mp4")

if not cap.isOpened():
    raise IOError("Cannot open video file")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Resolution: {width}x{height}")
print(f"FPS: {fps}")
print(f"Total frames: {frame_count}")

cap.release()

OSError: Cannot open video file

In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture("./crowd_5s.mp4")
ret, frame = cap.read()

if not ret:
    raise ValueError("Failed to read frame from video!")

results = model(frame, imgsz=1280)
boxes = results[0].boxes

annotated_frame = frame.copy()

for box in boxes:
    cls_id = int(box.cls.item())
    conf = float(box.conf.item())
    xyxy = box.xyxy[0].cpu().numpy()

    if cls_id == 0 and conf > 0.1:
        x1, y1, x2, y2 = map(int, xyxy)
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
        label = f"person {conf:.2f}"
        cv2.putText(
            annotated_frame,
            label,
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.5,
            color=(0, 255, 0),
            thickness=1,
            lineType=cv2.LINE_AA
        )

cv2.imwrite("annotated_frame1.jpg", annotated_frame)
cap.release()

print("✅ Frame processed and saved as 'annotated_frame.jpg'")

# видеопоток


Урежем видео для экспериметов

In [ ]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

# Open input video
input_video_path = "./crowd_5s.mp4"  # Убедитесь, что файл лежит в рабочей директории
capture = cv2.VideoCapture(input_video_path)

if not capture.isOpened():
    raise IOError(f"Cannot open video file: {input_video_path}")

# Get video properties
fps = int(capture.get(cv2.CAP_PROP_FPS))
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Используем AVI для надёжности
output_video_path = "../detect.avi"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Process each frame
while True:
    ret, frame = capture.read()
    if not ret:
        break

    # Run inference with higher resolution for better accuracy on 1080p video
    results = model(frame)
    boxes = results[0].boxes

    annotated_frame = frame.copy()

    for box in boxes:
        cls_id = int(box.cls.item())
        conf = float(box.conf.item())
        xyxy = box.xyxy[0].cpu().numpy()

        # Only detect 'person' (class ID 0 in COCO)
        if cls_id == 0 and conf > 0.3:  # reasonable confidence threshold
            x1, y1, x2, y2 = map(int, xyxy)
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
            label = f"person {conf:.2f}"
            cv2.putText(
                annotated_frame,
                label,
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.5,
                color=(0, 255, 0),
                thickness=1,
                lineType=cv2.LINE_AA,
            )

    writer.write(annotated_frame)

# Release resources
capture.release()
writer.release()

print(f"✅ Processing complete. Output saved as '{output_video_path}'")


In [ ]:
import subprocess
import os
subprocess.run(["ffmpeg", "-version"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

print("🔧 Converting to H.264 MP4 for compatibility...")
result = subprocess.run([
    "ffmpeg", "-y", "-i", output_video_path,
    "-c:v", "libx264", "-crf", "23", "-pix_fmt", "yuv420p", "-preset", "fast",
    output_video_path
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

## Try to use SASHI

In [ ]:
!pip install -U ultralytics sahi

In [ ]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
import cv2

# Load YOLOv8 model via SAHI
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path="yolov8m.pt",
    confidence_threshold=0.1,
    device="cpu"  # или "cuda:0", если есть GPU
)

In [ ]:
from ultralytics import YOLO
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
import cv2

# Load model via SAHI
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path="yolov8m.pt",
    confidence_threshold=0.27,
    device="cpu"
)

cap = cv2.VideoCapture("./crowd_5s.mp4")
ret, frame = cap.read()
if not ret:
    raise ValueError("Failed to read frame from video!")

# Run SAHI inference
result = get_sliced_prediction(
    image=frame,
    detection_model=detection_model,
    slice_height=512,
    slice_width=512,
    overlap_height_ratio=0.1,
    overlap_width_ratio=0.1,
    postprocess_match_threshold = 0.2
)


annotated_frame = frame.copy()

# Process each detected object
for obj in result.object_prediction_list:
    # Get class name and confidence
    class_name = obj.category.name  # e.g., "person"
    confidence = obj.score.value    # float

    # Only keep 'person'
    if class_name == "person":
        # Get bounding box in [x1, y1, x2, y2] format
        bbox = obj.bbox.to_voc_bbox()  # returns [x1, y1, x2, y2]
        x1, y1, x2, y2 = map(int, bbox)

        # Draw
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"person {confidence:.2f}"
        cv2.putText(
            annotated_frame,
            label,
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            1,
            cv2.LINE_AA
        )

cv2.imwrite("annotated_frame_sahi_new.jpg", annotated_frame)
cap.release()
print("✅ Frame processed with SAHI and saved as 'annotated_frame_sahi.jpg'")

In [ ]:
import cv2
from ultralytics import YOLO

# Load model via SAHI
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path="yolov8s.pt",
    confidence_threshold=0.27,
    device="cpu"
)



# Open input video
input_video_path = "./crowd_5s.mp4"  # Убедитесь, что файл лежит в рабочей директории
capture = cv2.VideoCapture(input_video_path)

if not capture.isOpened():
    raise IOError(f"Cannot open video file: {input_video_path}")

# Get video properties
fps = int(capture.get(cv2.CAP_PROP_FPS))
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Используем AVI для надёжности
output_video_path = "./detect_sahi.avi"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Process each frame
while True:
    ret, frame = capture.read()
    if not ret:
        break
    result = get_sliced_prediction(
        image=frame,
        detection_model=detection_model,
        slice_height=640,
        slice_width=640,
        overlap_height_ratio=0.1,
        overlap_width_ratio=0.1,
        postprocess_match_threshold = 0.2
    )
    # Run inference with higher resolution for better accuracy on 1080p video
    annotated_frame = frame.copy()

    for obj in result.object_prediction_list:
      # Get class name and confidence
      class_name = obj.category.name  # e.g., "person"
      confidence = obj.score.value    # float

      # Only keep 'person'
      if class_name == "person":
          # Get bounding box in [x1, y1, x2, y2] format
          bbox = obj.bbox.to_voc_bbox()  # returns [x1, y1, x2, y2]
          x1, y1, x2, y2 = map(int, bbox)

          cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
          label = f"person {conf:.2f}"
          cv2.putText(
              annotated_frame,
              label,
              (x1, y1 - 10),
              cv2.FONT_HERSHEY_SIMPLEX,
              fontScale=0.5,
              color=(0, 255, 0),
              thickness=1,
              lineType=cv2.LINE_AA,
          )

    writer.write(annotated_frame)

# Release resources
capture.release()
writer.release()

print(f"✅ Processing complete. Output saved as '{output_video_path}'")


In [ ]:
import subprocess
import os
final_output = "detect.mp4"
subprocess.run(
    [
        "ffmpeg",
        "-y",
        "-i", output_video_path,
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        "-preset", "fast",
        final_output
    ],
    check=True,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)